!conda install --file /content/drive/MyDrive/GitHub/Pytorch-Geometric-Neural-Automata/requirements.txt
import sys
sys.path.insert(0,'/content/drive/MyDrive/GitHub/Pytorch-Geometric-Neural-Automata/')

In [ ]:

# conda install pytorch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0 -c pytorch -y & conda install pyg -c pyg -y & conda install -c anaconda networkx -y & conda install -c conda-forge matplotlib -y
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import torch_geometric.utils as utils
from utils import *
from model import UpdateRule
from datasets import *


In [ ]:
seed(12345)
d_len = 4
dataset = load_arc(print_filenames=True)

def process_arc_sample(sample, io, final_dim=3, pad = False):
    sample = torch.tensor([x[io] for x in sample['train'] + sample['test']])
    if pad:
        sample = F.pad(sample, (0, final_dim - sample.shape[2], 0, final_dim - sample.shape[1]))
    sample = one_hot(sample.flatten(1,2), 10)[:d_len]
    return sample

datasets = [
        Dataset(
            process_arc_sample(sample, 'input'),
            process_arc_sample(sample, 'output'),
            
            metadata = sample["filename"]
        )
    for sample in dataset]
training_set = D(
    datasets[:-1]
)
training_set.batch_size = 1
testing_set = D(
    [datasets[-1]]
)

# for idx, i in enumerate(training_set):
#     print(idx)
#     print(torch.argmax(i[0], -1), torch.argmax(i[1], -1))
    

In [ ]:

n= 5


heads = 1
hidden_dim = 64
edge_dim = None
# edge_dim = 8

n_inputs = training_set.n_inputs
n_outputs = training_set.n_outputs
height = 3
width = training_set.n_inputs

cuda_device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
# cuda_device = torch.device("cpu")

update_rule = UpdateRule(
    n_inputs, 
    n_outputs,
    hidden_dim,
    edge_dim,
    network_width=hidden_dim,#hidden_dim,
    input_vector_size = 10,
    heads = heads,
    cuda_device=cuda_device
)
# update_rule.build_graph(
#     height,
#     width,
#     mode="grid",
#     input_mode="grid",
#     n_edge_switches=0
# )
update_rule.build_graph_3d(
    (3,3),
    height
)

update_rule = update_rule.to(cuda_device)
edge_attr = None

update_rule.draw()

#load initial_meta.pt
# update_rule.load_state_dict(torch.load("E52.pt", map_location=cuda_device))
seed(12345)

print(count_parameters(update_rule))


In [ ]:
from training import *
import time
n_steps = 6
batch_size = 32
training_set.batch_size = batch_size
lr = 0.001
optimizer = torch.optim.Adam(update_rule.parameters(), lr=lr)
use_wandb = True

wandb_name = "E54: pos encoding2"
if use_wandb:
    wandb.init(
        project="Experiments",
        name=wandb_name,
        # resume=True,
        tags=["Arc3x3", "batch_size=32"]
        )
    # wandb.run.log_code(".")

start_time = time.time()
train_on_meta_set(
    update_rule, optimizer, training_set, testing_set, {
        "n_steps": n_steps,
        "batch_size": batch_size,
        "n_epochs": 10000000,
        "n_edge_switches": 0,
    }, edge_attr=edge_attr,# edge_index=best_edges,
    # wandb_loss = f"baseline loss",
    # wandb_acc = f"baseline acc",
    wandb_log = use_wandb,
    save_dir=None,#"Edge_Weight",
    last_idx = 3,
    device=cuda_device,
)
# if use_wandb:
#     wandb.run.log_code(".")
#     wandb.finish()

print(f"Training took {time.time() - start_time} seconds")
assert False, "stop here"

In [ ]:
torch.save(update_rule.state_dict(), "E52.pt")

In [ ]:
test = datasets[6  ]
# test = Dataset(test_dataset.data[0:4], test_dataset.target[0:4])
# test = Dataset(test.data[0:4], test.target[0:4])

correct = test.target[-1].clone()
# test.target[-1] = torch.zeros_like(test.target[-1])

# data = D(datasets ,shuffle=False)
data = D([test] ,shuffle=False)
# data = training_set

update_rule.load_state_dict(torch.load("test3d.pt", map_location=cuda_device))

batch_size = 1

data.batch_size = batch_size

loader = DataLoader([update_rule.graph]*batch_size, batch_size = batch_size)
graph = loader.__iter__().__next__()
graph.batch = graph.batch.to(cuda_device)
x = update_rule.initial_state().repeat(batch_size, 1)
edge_index = update_rule.get_batch_edge_index(batch_size=batch_size, n_edge_switches=0)
update_rule.reset()

training_set.batch_size = batch_size



# training_set.batch_size=32
_, test_loss, network_output, correct, _, metadata = update_rule.eval()(
                x, 6, data, batch=graph.batch,
                edge_attr=edge_attr, edge_index=edge_index, last_idx = 3,
            )

torch.argmax(torch.tensor(network_output),-1)[0], torch.argmax(torch.tensor(correct),-1)[0], test_loss, metadata

In [ ]:
data = D([test] ,shuffle=False)
data.batch_size = 1
for i in data:
    print(torch.argmax(i[0], -1), torch.argmax(i[1], -1))

In [ ]:
#save update_rule
# torch.save(update_rule.state_dict(), "E51.pt")




In [ ]:
from model import *
count_parameters(SelfAttnAggregation(16, heads).attention1)


In [ ]:
edge_index = update_rule.get_batch_edge_index(batch_size=5, n_edge_switches=0)

graph = Data(edge_index=edge_index)
graph = utils.to_networkx(graph, to_undirected=True, remove_self_loops = True)
nx.draw(graph)

In [ ]:
datasets[:-1]

In [ ]:
for d in datasets:
    print(len(d))